# Parameter Estimation

Parameter estimation for ODE models is provided by the JuliaDiffEq suite. The current functionality includes build_loss_objective. Note that these require that the problem be defined using a [ParameterizedFunction](https://github.com/JuliaDiffEq/ParameterizedFunctions.jl):


## Setting up the parameter estimation problem

Let's try parameter estimation for an ODE on the challenging Lorenz attractor. We base our implementation on the code from Paulo Marques available at: https://github.com/pjpmarques/Julia-Modeling-the-World/blob/master/Lorenz%20Attractor.ipynb and on the global optimization method provided by BlackBoxOptim.

The system is formally described by three different differential equations. These equations represent the movement
of a point $(x, y, z)$ in space over time. In the following equations, $t$ represents time, $\sigma$, $\rho$, $\beta$ are constants.

$$ \begin{align}
    \frac{dx}{dt} &= \sigma (y - x) \\
    \frac{dy}{dt} &= x (\rho - z) - y \\
    \frac{dz}{dt} &= x y - bz
\end{align} $$


Or in matrix form:

$$ \begin{pmatrix}
\dot{x} \\
\dot{y} \\
\dot{z}
\end{pmatrix} = 
\begin{pmatrix}
- \sigma & \sigma & 0 \\
r - z & -1 & - x \\
y & x & -b
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
z
\end{pmatrix}$$

Initial values of the system in space:

True parameters: sigma, rho and beta are used to construct the dataset. In the estimation problem of the ODE system, these are unknown and should be estimated.

In [1]:
using DifferentialEquations, RecursiveArrayTools
using BlackBoxOptim

In [2]:
g1 = @ode_def_nohes LorenzExample begin
  dx = σ*(y-x)
  dy = x*(ρ-z) - y
  dz = x*y - β*z
end σ=>10.0 ρ=>28.0 β=>2.6666

r0 = [0.1; 0.0; 0.0]
tspan = (0.0, 4.0)
prob = ODEProblem(g1, r0, tspan)
tspan2 = (0.0, 3.0)
prob_short = ODEProblem(g1, r0, tspan2)

DiffEqBase.ODEProblem{Array{Float64,1},Float64,true,LorenzExample,DiffEqBase.CallbackSet{Tuple{},Tuple{}}}(LorenzExample,[0.1,0.0,0.0],(0.0,3.0),DiffEqBase.CallbackSet{Tuple{},Tuple{}}((),()))

The second phase of parameter estimation consists in forming the dataset by integrating the dynamic system over a time/space interval. 

Define the time vector and the interval grid

In [3]:
dt = 0.001
tf = 4.0
tinterval = 0:dt:tf
t  = collect(tinterval)

4001-element Array{Float64,1}:
 0.0  
 0.001
 0.002
 0.003
 0.004
 0.005
 0.006
 0.007
 0.008
 0.009
 0.01 
 0.011
 0.012
 ⋮    
 3.989
 3.99 
 3.991
 3.992
 3.993
 3.994
 3.995
 3.996
 3.997
 3.998
 3.999
 4.0  

But in order to compare to the parameter estimation in the [Xiang2015] paper, we also use their integration interval of 300 observations instead:

In [4]:
h = 0.01
M = 300
tstart = 0.0
tstop = tstart + M * h
tinterval_short = 0:h:tstop
t_short = collect(tinterval_short)

301-element Array{Float64,1}:
 0.0 
 0.01
 0.02
 0.03
 0.04
 0.05
 0.06
 0.07
 0.08
 0.09
 0.1 
 0.11
 0.12
 ⋮   
 2.89
 2.9 
 2.91
 2.92
 2.93
 2.94
 2.95
 2.96
 2.97
 2.98
 2.99
 3.0 

Integrate the ODE system from a starting point and into the future given a sequence of time steps.

In [5]:
#solve(prob_short,Euler(),tstops=t_short)
#data_short = vecarr_to_arr(solve(prob,Euler(),tstops=t_short))

In [6]:
sol = solve(prob_short,Vern7(),saveat=t_short,abstol=1e-12,reltol=1e-12)
data_short = vecarr_to_arr(sol)

LoadError: MethodError: no method matching vecarr_to_arr(::DiffEqBase.ODESolution{Array{Array{Float64,1},1},Void,Void,Array{Float64,1},Array{Array{Array{Float64,1},1},1},DiffEqBase.ODEProblem{Array{Float64,1},Float64,true,LorenzExample,DiffEqBase.CallbackSet{Tuple{},Tuple{}}},OrdinaryDiffEq.Vern7,OrdinaryDiffEq.InterpolationData{LorenzExample,Array{Array{Float64,1},1},Array{Float64,1},Array{Array{Array{Float64,1},1},1},OrdinaryDiffEq.Vern7Cache{Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},OrdinaryDiffEq.Vern7ConstantCache{Float64,Float64}}}})[0m
Closest candidates are:
  vecarr_to_arr([1m[31m::RecursiveArrayTools.AbstractVectorOfArray{T,N}[0m) at C:\Users\Denis\.julia\v0.5\RecursiveArrayTools\src\vector_of_array.jl:44[0m

Calculate the actual/original state vectors that will be used for parameter estimation:

In [ ]:
#solve(prob,Euler(),tstops=t)
#data = vecarr_to_arr(solve(prob,Euler(),tstops=t))

In [ ]:
sol = solve(prob,Vern7(),saveat=t,abstol=1e-12,reltol=1e-12)
data = vecarr_to_arr(sol)

The fundamental layout of the data matrix is: equations as rows and sequence of observations as columns:

In [ ]:
display(data)


## Non Linear Programming as an estimation method

The third phase of the parameter estimation problem is to set up the objective function of the NLP problem. The solution below should then be matched with build_loss_objective to simplify and standardize the NLP problem. The aim of build_loss_objective is to make the NLP problem compatible with solvers like Optim.jl and MathProgBase-associated solvers like NLopt.

We have n observations $(\boldsymbol{x}_i, y_i), i = 1,2...,n$, from a fixed-regressor nonlinear model with a known functional relationship $f$. Thus

$$ \begin{align}
        y_i = f(\boldsymbol{x}_i; \boldsymbol{\theta}^*) + \epsilon_i & \quad (i = 1,2,....,n)
\end{align} $$

where $E[\epsilon_i]$ = 0, $\boldsymbol{x}_i$ is a k x 1 vector, and the true value $\boldsymbol{\theta}^*$ of $\boldsymbol{\theta}$ is known to belong to $\boldsymbol{\Theta}$, a subset of $R^p$. The nonlinear least squares estimate of $\boldsymbol{\theta}^*$, denoted by $\boldsymbol{\hat{\theta}}$, minimizes the error sum of squares:

$$ \begin{align}
S(\boldsymbol{\theta}) = \sum_{i=1}^{n} [y_i - f(\boldsymbol{x}_i;\boldsymbol{\theta})]^2 & \quad   \boldsymbol{\theta} \in \boldsymbol{\Theta}.
\end{align} $$

With the notation $f_i(\boldsymbol{\theta})$ = $f(\boldsymbol{x}_i;\boldsymbol{\theta})$ and $\boldsymbol{f}(\boldsymbol{\theta})$ = $\big($$f_1(\boldsymbol{\theta})$, $f_2(\boldsymbol{\theta})$,$\dots$, $f_n(\boldsymbol{\theta})$$\big)$', the error sum of squares, $S(\boldsymbol{\theta})$, can also be re-written as:

$$ \begin{align}
S(\boldsymbol{\theta}) = [y - \boldsymbol{f}(\boldsymbol{\theta})]'[y - \boldsymbol{f}(\boldsymbol{\theta})] = \|y - \boldsymbol{f}(\boldsymbol{\theta}\|^2
\end{align} $$

This quantity, labelled the data misfit function, is a quadratic metric, squared Euclidean distance,  named ESS = Error Sum of Squares, columnwise. This is the $S(\boldsymbol{\theta})$ function 2.2 and 2.7 of Seber and Wild page 21.

In [ ]:
# build cost function
function ess2(actual, estimated)
    sumsq = 0.0
    for i in 1:size(actual, 2)
        @inbounds sumsq += sumabs2(actual[i] .- estimated[:, i])
    end
    sumsq
end

In [ ]:
my_cost_short(sol) = ess2(sol,data_short)
my_cost(sol) = ess2(sol,data)

A typical problem is to find the value of $\boldsymbol{\theta}$ that minimizes a data misfit/loss function $\boldsymbol{h(\theta)}$ having one of the following forms:

$$ \begin{align}
h(\boldsymbol{\theta}) = \sum_{i=1}^{n} [y_i - f(\boldsymbol{x}_i;\boldsymbol{\theta})]^2 = & \sum_{i=1}^{n} r_i(\boldsymbol{\theta})^2,
\end{align} $$

$$ \begin{align}
h(\boldsymbol{\theta}) = \sum_{i=1}^{n} |y_i - f(\boldsymbol{x}_i;\boldsymbol{\theta})| = & \sum_{i=1}^{n} |r_i(\boldsymbol{\theta})|,
\end{align} $$

$$ \begin{align}
h(\boldsymbol{\theta}) = \sum_{i=1}^{n} \rho(r_i(\boldsymbol{\theta})) & \qquad \text{(robust loss functions)}.
\end{align} $$

These functions are available from two Julia packages: Distances.jl and LossFunctions.jl on Github. The ess function has been specified from first principles to check the accuracy and speed of alternative functions from other packages.

We restrict attention to unconstrained minimization where we wish to minimize $h(\boldsymbol{\theta})$ over $\mathbb{R}^p$. Most estimation methods require a global minimum of $h(\boldsymbol{\theta})$, namely a point $\boldsymbol{\hat{\theta}}$ such that $h(\boldsymbol{\theta}) \geq h(\boldsymbol{\hat{\theta}})$ for all $\boldsymbol{\theta}$ in $\mathbb{R}^p$. Global minimization is possible only for very restrictive clases of functions such as convex functions (Dixon and Szeg$\ddot{o}$ [1978]. Therefore, the properties of the selected data misfit functions should check that they are, first and foremost, convex and subsidiarily smooth and/or continuous. 

### Nonlinear Least Squares estimation (nls)

A family of NLS estimators covers several cases of generalized or weighted least squares estimators. The main steps leading to the computation of $\boldsymbol{\hat{\theta}}_G$, the value of $\boldsymbol{\theta}$ minimizing:

$$ \begin{align}
S(\boldsymbol{\theta}) = [y - \boldsymbol{f}(\boldsymbol{\theta})]'\boldsymbol{V^{-1}}[y - \boldsymbol{f}(\boldsymbol{\theta})],
\end{align} $$

$\boldsymbol{V}$ is a known positive definite matrix, $\boldsymbol{J}$ is the Jacobian of $\boldsymbol{f}(\boldsymbol{\theta})$ and $\mathscr{\hat{D}}[\boldsymbol{\hat{\theta}}_G] = \hat{\sigma}^2(\boldsymbol{\hat{J}}'\boldsymbol{V}^{-1}\boldsymbol{\hat{J}})^{-1}$           , an estimate of the asymptotic covariance matrix of $\boldsymbol{\hat{\theta}}_G$:
1. Perform a Cholesky decomposition of $\boldsymbol{V} = \boldsymbol{U}'\boldsymbol{U}$ of $\boldsymbol{V}$
2. Solve $U'z = y$ and $U'k(\boldsymbol{\theta})$ for $\boldsymbol{z}$ and $\boldsymbol{k}$
3. Apply an ordinary nonlinear least-squares technique to

$$ \begin{align}
[z - \boldsymbol{k}(\boldsymbol{\theta})]'[z - \boldsymbol{k}(\boldsymbol{\theta})],
\end{align} $$

Here we illustrate the simplest case when $V = I_n$ and the above steps collapse to an OLS.

### Maximum-likelihood estimation (mle)

As in the NLS case, the family of likelihood estimators covers several cases: maximum likelihood estimator, concentrated likelihood estimator and quasi-likelihood estimator.

If the joint distribution of the $\epsilon_i$ in the above model is assumed to be a known variable instead of a constant, then the maximum-likelihood estimate of $\boldsymbol{\theta}$ is obtained by maximizing the likelihood function. For $\epsilon_i$ i.i.d $N(0, \sigma^2)$ and ignoring some constants then the logarithm of the likelihood $L(\boldsymbol{\theta}, \sigma^2)$ is:

$$L(\boldsymbol{\theta}, \sigma^2) = - \frac{n}{2} ln  \sigma^2 - \frac{1}{2\sigma^2}S(\boldsymbol{\theta})$$

This function is maximized with respect to $\boldsymbol{\theta}$ when $S(\boldsymbol{\theta})$ is minimized when $\boldsymbol{\theta} = \boldsymbol{\hat{\theta}}$, the least squares estimate.

By analogy to the family of least squares estimators above, it is envisaged that this family of likelihood estimators should be coded as follows: 

For the additional cases: the concentrated likelihood and the quasi-likelihood estimators. Each case depends on the properties of the covariance matrix of experimental noise.

The concentrated likelihood method is a two-step method where the first step is the simple likelihood method explained above. It is documented on page 36 of Seber and Wild.

The quasi-likelihood method is a further refinement by relaxing the underlying assumptions of MLE. It is documented on page 43 of Seber and Wild. 

## Global optimization using BlackBoxOptim

The parameter estimation of the Lorenz atractor has proved inaccurate with the family of algorithms available in NLopt. Instead, we are attempting the estimation of these parameters within the class of Differential Evolution (DE) algorithm available with BlackBoxOptim (https://github.com/robertfeldt/BlackBoxOptim.jl). BlackBoxOptim does not yet comply with MathProgBase syntax and the optimization has been set up by Robert Feldt below. It will become MathProgBase compliant in the near future.

To demonstrate the estimation capabilities of BlackBoxOptim, let's sample a small subset first and set the estimation bounds of the parameters as in the [Xiang2015] paper, https://www.hindawi.com/journals/ddns/2015/740721/:

In [ ]:
Xiang2015Bounds = Tuple{Float64, Float64}[(9, 11), (20, 30), (2, 3)]

In [ ]:
obj_short = build_loss_objective(prob_short,Euler(),my_cost_short,tstops=t_short,dense=false)
res1 = bboptimize(obj_short;SearchRange = Xiang2015Bounds, MaxSteps = 11e3)

In [ ]:
obj = build_loss_objective(prob,Euler(),my_cost,tstops=t,dense=false)
res1 = bboptimize(obj;SearchRange = Xiang2015Bounds, MaxSteps = 8e3)

But let's also try relaxing the tight bounds inthe Xiang2015 paper.

In [ ]:
LooserBounds = Tuple{Float64, Float64}[(0, 22), (0, 60), (1, 6)]

In [ ]:
res3 = bboptimize(params -> nls_estimation(params, datastates1_short, times1_short); 
    SearchRange = LooserBounds, MaxSteps = 11e3) # They allow 12k fitness evals for 3-param estimation

In [ ]:
println("Results from the short data sequence, 300 observations, used in [Xiang2015] paper:")
estfitness = nls_estimation(best_candidate(res2), datastates_short, t_short)
@show (estfitness, best_candidate(res2), best_fitness(res1))
datafitness = nls_estimation(true_params, datastates_short, t_short)
@show (datafitness, true_params)

In [ ]:
println("Results from the long data sequence from Paulo Marques:")
estfitness = nls_estimation(best_candidate(res1), datastates, t)
@show (estfitness, best_candidate(res1), best_fitness(res2))
datafitness = nls_estimation(true_params, datastates, t)
@show (datafitness, true_params)

In [ ]:
println("Results from the short data sequence used in [Xiang2015] paper, but with looser bounds:")
estfitness = nls_estimation(best_candidate(res3), datastates_short, t_short)
@show (estfitness, best_candidate(res3), best_fitness(res3))
datafitness = nls_estimation(true_params, datastates_short, t_short)
@show (datafitness, true_params)